In [2]:
import pandas as pd
import os
import re
import numpy as np
from bs4 import BeautifulSoup
import Scraper as scraper
import collections
import itertools
from os import listdir
from os.path import isfile, join
import lxml.html


In [3]:
data = pd.read_csv('list_of_companies_sector_15_with_new_ratios.csv')
n = len(data)
data['d_RoE'] = np.zeros(n)
data['d_Net_Profit_Margin'] = np.zeros(n)
data['d_Asset_turnover'] = np.zeros(n)
data['d_Financial_leverage'] = np.zeros(n)

In [4]:
data.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,costat,sic,RoE,Net_Profit_Margin,Asset_turnover,Financial_leverage,d_RoE,d_Net_Profit_Margin,d_Asset_turnover,d_Financial_leverage
0,1009,19901031,1990,INDL,C,D,STD,ABSI,000781104,ABS INDUSTRIES INC,...,I,3460,26.820146,0.105696,1.253193,5.163686,0,0,0,0
1,1009,19911031,1991,INDL,C,D,STD,ABSI,000781104,ABS INDUSTRIES INC,...,I,3460,22.588790,0.095384,1.018645,5.438819,0,0,0,0
2,1009,19921031,1992,INDL,C,D,STD,ABSI,000781104,ABS INDUSTRIES INC,...,I,3460,39.219675,0.101251,1.224676,5.570803,0,0,0,0
3,1009,19931031,1993,INDL,C,D,STD,ABSI,000781104,ABS INDUSTRIES INC,...,I,3460,76.693946,0.101893,1.130678,6.152375,0,0,0,0
4,1009,19941031,1994,INDL,C,D,STD,ABSI,000781104,ABS INDUSTRIES INC,...,I,3460,136.622360,0.103602,0.981996,6.553336,0,0,0,0


In [5]:
count = 0
gvkey = ''
for i, row in data.iterrows():
    if row['gvkey'] != gvkey:
        count += 1
        gvkey = row['gvkey']
    else:
        last_row = data.iloc[i - 1, :]
        if row['fyear'] - last_row['fyear'] == 1:
            data.loc[i,'d_RoE'] = row['RoE'] - last_row['RoE']
            data.loc[i,'d_Net_Profit_Margin'] = row['Net_Profit_Margin'] - last_row['Net_Profit_Margin']
            data.loc[i,'d_Asset_turnover'] = row['Asset_turnover'] - last_row['Asset_turnover']
            data.loc[i,'d_Financial_leverage'] = row['Financial_leverage'] - last_row['Financial_leverage']

In [7]:
data.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,costat,sic,RoE,Net_Profit_Margin,Asset_turnover,Financial_leverage,d_RoE,d_Net_Profit_Margin,d_Asset_turnover,d_Financial_leverage
0,1009,19901031,1990,INDL,C,D,STD,ABSI,000781104,ABS INDUSTRIES INC,...,I,3460,26.820146,0.105696,1.253193,5.163686,0.000000,0.000000,0.000000,0.000000
1,1009,19911031,1991,INDL,C,D,STD,ABSI,000781104,ABS INDUSTRIES INC,...,I,3460,22.588790,0.095384,1.018645,5.438819,-4.231356,-0.010312,-0.234548,0.275133
2,1009,19921031,1992,INDL,C,D,STD,ABSI,000781104,ABS INDUSTRIES INC,...,I,3460,39.219675,0.101251,1.224676,5.570803,16.630885,0.005867,0.206031,0.131984
3,1009,19931031,1993,INDL,C,D,STD,ABSI,000781104,ABS INDUSTRIES INC,...,I,3460,76.693946,0.101893,1.130678,6.152375,37.474271,0.000643,-0.093998,0.581572
4,1009,19941031,1994,INDL,C,D,STD,ABSI,000781104,ABS INDUSTRIES INC,...,I,3460,136.622360,0.103602,0.981996,6.553336,59.928414,0.001708,-0.148682,0.400961


In [8]:
data.to_csv('deltas.csv')

In [9]:
#Importing the list of company names from SEC that correspond to our sector
sector_code='15'
tic_table=pd.read_table('tic_company_gics.txt')
tic_code=tic_table[tic_table.gsector==int(sector_code)]
tic_code=tic_code.drop_duplicates()

tic_code.head()
list_names = tic_code.tic

data_filtered = data[data.tic.isin(list_names)]

In [10]:
files_names = [f for f in listdir('downloads-10K') if isfile(join('downloads-10K', f))]

In [11]:
files_names[0][:-6]
#'{}_{}'.format('tic', 'year')

'AA_2001'

In [12]:
data['filename'] = 0

def find_filename(row):
    try:
        tic = row.tic
        year = row.fyear
        matchs = [x for x in files_names if x[:-6] == '{}_{}'.format(tic, int(year))]
        return matchs[0]
    except:
        return 0


In [13]:
data['filename'] = data.apply(find_filename, axis=1)

In [14]:
data_filtered = data[data.filename != 0]

In [15]:
data_filtered.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,sic,RoE,Net_Profit_Margin,Asset_turnover,Financial_leverage,d_RoE,d_Net_Profit_Margin,d_Asset_turnover,d_Financial_leverage,filename
55,1209,19970930,1997,INDL,C,D,STD,APD,009158106,AIR PRODUCTS & CHEMICALS INC,...,2810,1818185.46,0.148044,0.640218,2.735584,389615.46,0.009571,0.025682,0.201784,APD_1997-12-11
56,1209,19980930,1998,INDL,C,D,STD,APD,009158106,AIR PRODUCTS & CHEMICALS INC,...,2810,2170648.74,0.165440,0.656777,2.807933,352463.28,0.017396,0.016560,0.072349,APD_1998-12-16
57,1209,19990930,1999,INDL,C,D,STD,APD,009158106,AIR PRODUCTS & CHEMICALS INC,...,2810,2134128.96,0.143543,0.609568,2.780760,-36519.78,-0.021897,-0.047209,-0.027173,APD_1999-12-16
58,1209,20000930,2000,INDL,C,D,STD,APD,009158106,AIR PRODUCTS & CHEMICALS INC,...,2810,2351835.68,0.152476,0.661036,2.931450,217706.72,0.008933,0.051468,0.150690,APD_2000-12-18
59,1209,20010930,2001,INDL,C,D,STD,APD,009158106,AIR PRODUCTS & CHEMICALS INC,...,2810,2489298.70,0.140191,0.707215,2.602904,137463.02,-0.012285,0.046179,-0.328546,APD_2001-12-14


In [16]:
def extract_MDA(df):
    # Credits for the scraper package go to GROUP 7
    mda_text = ''
    print(df.name)
    print('OK')
    with open('downloads-10K/' + df['filename']) as file:
        root = lxml.html.document_fromstring(file.read())
        soup = root.text_content()
        
        # We first try the scrape by regex method
        try:
            mda_text = scraper.scrapeByRegex(soup)
            if mda_text:
                mda_text = BeautifulSoup(mda_text, "html.parser").get_text()
                mda_text = re.sub('[^\w]', ' ', mda_text)
                mda_text = re.sub("\d+","",mda_text)
                return mda_text
        except:
            pass

In [20]:
subset = data_filtered.iloc[:5,:]
subset['MDA'] = subset.apply(extract_MDA,1)
#subset.head()

55
OK
56
OK
57
OK
58
OK
59
OK


/Users/martingouy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
